In [2]:
import pandas as pd
import numpy as np
import time
import itertools as it

from fun_ideal import calc_flood_height, generate_groups, calc_group_vol, calc_group_h
from ideal_damage import Damage
from topo_ideal import Topo

In [5]:
topo_file = "ideal_topo_gp_points_1.csv"
storm_file = "SurgeData/surge_w.csv"
div_data_file = "ideal_div_data.csv"
time_file = "SurgeData/time_w.csv"
damage_table_file = "damage_table.csv"

In [6]:
surge = pd.read_csv(storm_file).values
surge_time = pd.read_csv(time_file).values
damage_table = pd.read_csv(damage_table_file)



Topo = Topo()
Damage = Damage(damage_table_file)

In [11]:
# generate groups
groups = generate_groups(Topo.div_data)


divs_allocate = list(range(11,17))
h_start = 0
h_end = 5
h_num = 10
segment_l = 100
wall_height = np.zeros(Topo.shore_height.size)

positions = []
for i, p in enumerate(wall_height):
    if Topo.all_divs[i] in divs_allocate:
        positions.append(i)

In [12]:
points = []
wall_height[positions] = h_start

for h in np.linspace(h_start, h_end, num=h_num):
    wall_height[:] = 0
    wall_height[positions] = h
    
    height_div, volume_div = calc_flood_height(Topo, surge, surge_time, wall_height)
    # calculate group volumes
    volume_grouped = calc_group_vol(groups, volume_div)
    # get height in each group
    height_group = calc_group_h(Topo, groups, volume_grouped)
    
    #Damage
    dmg = Damage.calc_damage(height_group)
    cost_dmg = np.sum(dmg)
    cost_wall = np.sum(49212 * wall_height * segment_l)
    cost_tot = cost_dmg + cost_wall

    points.append([h, cost_wall, cost_dmg, cost_tot])
    
df = pd.DataFrame(points)
print(df)

          0             1             2             3
0  0.000000  0.000000e+00  1.226800e+08  1.226800e+08
1  0.555556  1.312320e+08  8.905358e+07  2.202856e+08
2  1.111111  2.624640e+08  8.316970e+07  3.456337e+08
3  1.666667  3.936960e+08  8.017305e+07  4.738691e+08
4  2.222222  5.249280e+08  8.017305e+07  6.051011e+08
5  2.777778  6.561600e+08  8.017305e+07  7.363331e+08
6  3.333333  7.873920e+08  8.017305e+07  8.675651e+08
7  3.888889  9.186240e+08  8.017305e+07  9.987971e+08
8  4.444444  1.049856e+09  8.017305e+07  1.130029e+09
9  5.000000  1.181088e+09  8.017305e+07  1.261261e+09
